In [ ]:
%load_ext autoreload
%autoreload 2

from opengsync_db import categories, DBHandler
from IPython.display import display
import pandas as pd
from dotenv import load_dotenv
import os
load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [23]:
db = DBHandler()
db.connect(
    user=os.environ["POSTGRES_USER"],
    password=os.environ["POSTGRES_PASSWORD"],
    host=os.environ["POSTGRES_SERVER_IP"],
    port=os.environ["POSTGRES_PORT"],
    db=os.environ["POSTGRES_DB"],
)

LOG: Connected to DB 'postgresql+psycopg://localhost:5432/opengsync_db'


### 0. Categories (Enums)

In [19]:
# These are used to store statuses or types, e.g. LibraryType, ExperimentStatus, GenomeRef
# Each enum has a unique id inside it's own enum-class
# these are defined in services/opengsync-app/opengsync-db/opengsync_db/categories/
categories.LibraryType.POLY_A_RNA_SEQ.id, categories.LibraryType.POLY_A_RNA_SEQ.name, categories.GenomeRef.COVID.organism_tax_id

(101, 'Poly-A RNA-Seq', 2697049)

### 1. Get All Libraries from Sequencing Run (Before demultiplexing flowcell)

In [20]:
experiment = db.get_experiment(name="BSF_1764")
print(experiment)
print(experiment.flowcell_type)
print(experiment.operator)
print(experiment.num_lanes)
print(experiment.status)

Experiment(id=18, name=BSF_1764, num_lanes=2)
S1 [2]
User(id=3, email=csuete@cemm.at)
2
Archived [10]


In [21]:
# This dataframe should be sufficient to demultiplex a flowcell
experiment_libraries = db.get_flowcell_df(experiment_id=experiment.id)
experiment_libraries.sample(10)

,lane,sample_name,library_name,library_type,reference,seq_request_id,sequence_i7,sequence_i5
25,1,B2_HC_D_CD4,B2_HC_D_CD4_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,ACTCGCTA,TCTACTCT
91,1,B4_RA_R_Bcell,B4_RA_R_Bcell_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,GCTACGCT,ACTCTAGG
43,1,B2_RA_P_Monocytes,B2_RA_P_Monocytes_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,TAGCGCTC,CTCCTTAC
14,1,B1_RA_G_CD8,B1_RA_G_CD8_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,ACTCGCTA,AGAGGATA
144,2,15_5,15_5_SMARTSEQ,SMART-Seq [102],Human (GRCh38) [1],28,AGGCAGAA,GTAAGGAG
45,1,B2_RA_R_CD8,B2_RA_R_CD8_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,CCTAAGAC,CTCCTTAC
115,1,B5_RA_R_Bcell,B5_RA_R_Bcell_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,TAGCGCTC,CTAGTCGA
69,1,B3_RA_R_CD8,B3_RA_R_CD8_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,AAGAGGCA,CTAGTCGA
116,1,B5_RA_R_CD4,B5_RA_R_CD4_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,ACTGAGCG,CTAGTCGA
42,1,B2_RA_P_CD8,B2_RA_P_CD8_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,ATGCGCAG,CTCCTTAC


#### 1.1 Grouping by Sequencing Request

In [22]:
# Demultiplexed fastq-files should be grouped by 'seq_request_id' as each Sequencing Request can have only one Requestor
for (seq_request_id), df in experiment_libraries.groupby("seq_request_id"):
    print(f"seq_request_id: {seq_request_id}")
    display(df.head(20))
    print("...\n")


seq_request_id: 28


,lane,sample_name,library_name,library_type,reference,seq_request_id,sequence_i7,sequence_i5
119,2,07_13,07_13_SMARTSEQ,SMART-Seq [102],Human (GRCh38) [1],28,TAAGGCGA,CTCTCTAT
120,2,07_14,07_14_SMARTSEQ,SMART-Seq [102],Human (GRCh38) [1],28,CGTACTAG,CTCTCTAT
121,2,07_15,07_15_SMARTSEQ,SMART-Seq [102],Human (GRCh38) [1],28,AGGCAGAA,CTCTCTAT
122,2,07_16,07_16_SMARTSEQ,SMART-Seq [102],Human (GRCh38) [1],28,TCCTGAGC,CTCTCTAT
123,2,13_1,13_1_SMARTSEQ,SMART-Seq [102],Human (GRCh38) [1],28,GGACTCCT,CTCTCTAT
124,2,13_10,13_10_SMARTSEQ,SMART-Seq [102],Human (GRCh38) [1],28,TAGGCATG,CTCTCTAT
125,2,13_11,13_11_SMARTSEQ,SMART-Seq [102],Human (GRCh38) [1],28,CTCTCTAC,CTCTCTAT
126,2,13_12,13_12_SMARTSEQ,SMART-Seq [102],Human (GRCh38) [1],28,CGAGGCTG,CTCTCTAT
127,2,13_2,13_2_SMARTSEQ,SMART-Seq [102],Human (GRCh38) [1],28,AAGAGGCA,CTCTCTAT
128,2,13_3,13_3_SMARTSEQ,SMART-Seq [102],Human (GRCh38) [1],28,GTAGAGGA,CTCTCTAT


...

seq_request_id: 60


,lane,sample_name,library_name,library_type,reference,seq_request_id,sequence_i7,sequence_i5
0,1,B1_HC_D_Bcell,B1_HC_D_Bcell_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,GCTCATGA,ATAGAGAG
1,1,B1_HC_D_CD4,B1_HC_D_CD4_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,ATCTCAGG,ATAGAGAG
2,1,B1_HC_D_CD8,B1_HC_D_CD8_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,ACTCGCTA,ATAGAGAG
3,1,B1_HC_D_Monocytes,B1_HC_D_Monocytes_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,GGAGCTAC,ATAGAGAG
4,1,B1_RA_A_Bcell,B1_RA_A_Bcell_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,GCGTAGTA,ATAGAGAG
5,1,B1_RA_A_CD4,B1_RA_A_CD4_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,CGGAGCCT,ATAGAGAG
6,1,B1_RA_A_CD8,B1_RA_A_CD8_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,TACGCTGC,ATAGAGAG
7,1,B1_RA_A_Monocytes,B1_RA_A_Monocytes_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,ATGCGCAG,ATAGAGAG
8,1,B1_RA_D_Bcell,B1_RA_D_Bcell_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,TAGCGCTC,ATAGAGAG
9,1,B1_RA_D_CD4,B1_RA_D_CD4_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,ACTGAGCG,ATAGAGAG


...



#### 1.2 Filtering Library Type

In [23]:
experiment_libraries[experiment_libraries["library_type"] == categories.LibraryType.ATAC_SEQ].head(3)

,lane,sample_name,library_name,library_type,reference,seq_request_id,sequence_i7,sequence_i5
0,1,B1_HC_D_Bcell,B1_HC_D_Bcell_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,GCTCATGA,ATAGAGAG
1,1,B1_HC_D_CD4,B1_HC_D_CD4_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,ATCTCAGG,ATAGAGAG
2,1,B1_HC_D_CD8,B1_HC_D_CD8_ATAC,ATAC-Seq [108],Human (GRCh38) [1],60,ACTCGCTA,ATAGAGAG


### 2. Downstream Projects (after demultiplexing)

In [24]:
db.get_project_samples_df(23)

,sample_id,sample_name,sex,tissue,cell_type
0,556,07_13,f,Skin,Fibroblasts
1,557,07_14,f,Skin,Fibroblasts
2,558,07_15,f,Skin,Fibroblasts
3,559,07_16,f,Skin,Fibroblasts
4,560,13_1,f,Skin,Fibroblasts
5,561,13_10,f,Skin,Fibroblasts
6,562,13_11,f,Skin,Fibroblasts
7,563,13_12,f,Skin,Fibroblasts
8,564,13_2,f,Skin,Fibroblasts
9,565,13_3,f,Skin,Fibroblasts


In [25]:
# this dataframe should be enough to find all fastq-files generated from demultiplexing step
# projects can contain multiple requests, and libraries from multiple experiments
project_libraries = db.get_project_libraries_df(23)
project_libraries[["experiment_name", "lane", "sample_name", "sample_id", "pool_name", "library_name", "seq_request_id", "sex", "tissue", "cell_type"]].sample(10)

,experiment_name,lane,sample_name,sample_id,pool_name,library_name,seq_request_id,sex,tissue,cell_type
18,BSF_1764,2,14_3,574,20250422_SS2_DF_cyt_AK,14_3_SMARTSEQ,28,f,Skin,Fibroblasts
25,BSF_1764,2,15_5,581,20250422_SS2_DF_cyt_AK,15_5_SMARTSEQ,28,m,Skin,Fibroblasts
26,BSF_1764,2,15_6,582,20250422_SS2_DF_cyt_AK,15_6_SMARTSEQ,28,m,Skin,Fibroblasts
7,BSF_1764,2,13_12,563,20250422_SS2_DF_cyt_AK,13_12_SMARTSEQ,28,f,Skin,Fibroblasts
36,BSF_1764,2,18_4,592,20250422_SS2_DF_cyt_AK,18_4_SMARTSEQ,28,f,Skin,Fibroblasts
10,BSF_1764,2,13_4,566,20250422_SS2_DF_cyt_AK,13_4_SMARTSEQ,28,f,Skin,Fibroblasts
33,BSF_1764,2,18_1,589,20250422_SS2_DF_cyt_AK,18_1_SMARTSEQ,28,f,Skin,Fibroblasts
14,BSF_1764,2,13_8,570,20250422_SS2_DF_cyt_AK,13_8_SMARTSEQ,28,f,Skin,Fibroblasts
22,BSF_1764,2,15_1,578,20250422_SS2_DF_cyt_AK,15_1_SMARTSEQ,28,m,Skin,Fibroblasts
15,BSF_1764,2,13_9,571,20250422_SS2_DF_cyt_AK,13_9_SMARTSEQ,28,f,Skin,Fibroblasts


In [26]:
db.get_seq_requestor_df(28)

,seq_request_name,user_id,email,first_name,last_name,role_id,role
0,Stary lab pooled sample SrcX Fibroblast Project,3,csuete@cemm.at,Carina,Suete,3,Technician [3]


### 3. More advanced usage using Class hierarchy

In [27]:
db.open_session()
seq_request = db.get_seq_request(28)
print(seq_request.contact_person)
print(seq_request.comments)
print(seq_request.data_delivery_mode)
db.close_session()

Contact(id=107, name=Aglaja Kopf, email=akopf@cemm.oeaw.ac.at, phone=123, address=None)
[Comment(id=40, text=Request Accepted ✅, timestamp=2025-05-06 15:50:59.792390+02:00, author_id=3)]
Custom [0]


In [28]:
db.open_session()
project = db.get_project(23)
print(project.owner)
db.close_session()

User(id=3, email=csuete@cemm.at)
